In [1]:

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split



In [2]:
# import torch  
# import numpy as np
# x_train = np.load(f'/data3/chengjingwen/diffusion-mdpnet/data/lo/uv.npy')
# x_train = torch.tensor(x_train, dtype=torch.float32) 

# xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最小值
# xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最大值


# data = (x_train - xmin) / (xmax - xmin)
# data = data[:,:,0:1,:,:]   # (num_tra, steps, 1, 64, 64)

# # 滑动窗口划分 (groups, 6, 1, 64, 64)
# pred_step = 5
# num_trajectories, steps, num_var, x_dim, y_dim = data.shape
# groups_per_trajectory = steps - pred_step  
# groups = num_trajectories * groups_per_trajectory  


# # 使用 sliding_window_view 进行窗口切分
# data_groups = data.unfold(dimension=1, size=pred_step + 1, step=1)  # (num_trajectories, groups_per_trajectory, 6, 1, 64, 64)

# data_groups = data_groups.permute(0,1,5,2,3,4)

# data_groups = data_groups.reshape(-1, pred_step + 1, num_var, x_dim, y_dim)  # (groups, 6, 1, 64, 64)


# num_train = int(0.8 * groups)  # 80% 训练集
# num_val = groups - num_train

# indices = torch.randperm(groups)  # 生成随机排列索引
# train_indices = indices[:num_train]  
# val_indices = indices[num_train:]
# ds_train = data_groups[train_indices]
# ds_val = data_groups[val_indices]

# # 打印结果
# print(f"ds_train.shape: {ds_train.shape}")  # 预期: (train_groups, 6, 1, 64, 64)
# print(f"ds_val.shape: {ds_val.shape}")  # 预期: (val_groups, 6, 1, 64, 64)


ds_train.shape: torch.Size([7200, 6, 1, 64, 64])
ds_val.shape: torch.Size([1800, 6, 1, 64, 64])


In [2]:
import torch
import numpy as np


x_train = np.load(f'/data3/chengjingwen/diffusion-mdpnet/data/lo/uv.npy')
x_train = torch.tensor(x_train, dtype=torch.float32) 
print(x_train.shape)
# 计算训练集的 min 和 max，维度为 (1, n_vars, 1, 1)
xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最小值
xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最大值


data = (x_train - xmin) / (xmax - xmin)


# print(f"Training data normalized: min={data.min().item()}, max={data.max().item()}")


# torch.save({'xmin': xmin, 'xmax': xmax}, f'/data4/chengjingwen/kdd25-main 3/data/lo/normalization_params.pth')

torch.Size([100, 100, 2, 128, 128])


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class MultiStepInputDataset(Dataset):
    def __init__(self, data, history_steps=10):
        """
        data: Tensor, shape (num_trajectories, num_steps, num_variables, x_dim, y_dim)
        history_steps: how many past steps to use as input
        """
        self.data = data
        self.history_steps = history_steps
        self.num_trajectories, self.num_steps, self.num_variables, self.x_dim, self.y_dim = data.shape

        # 确保有足够时间步能取 history_steps + 1（因为input要10步，output是第11步）
        if self.num_steps <= self.history_steps:
            raise ValueError("num_steps must be greater than history_steps")

    def __len__(self):
        # 每条轨迹生成 (num_steps - history_steps) 个样本
        return self.num_trajectories * (self.num_steps - self.history_steps)

    def __getitem__(self, idx):
        # 确定轨迹编号和在轨迹内的起点时间步
        trajectory_idx = idx // (self.num_steps - self.history_steps)
        time_idx = idx % (self.num_steps - self.history_steps)

        # input: 连续 history_steps 个时间步，堆叠成channel
        input_data = self.data[trajectory_idx, time_idx : time_idx + self.history_steps]  # (history_steps, num_variables, x_dim, y_dim)
        

        # target: 预测的第 history_steps 后的一个时间步
        target_data = self.data[trajectory_idx, time_idx + self.history_steps].unsqueeze(0)  # (1, num_variables, x_dim, y_dim)

        return input_data, target_data


# train_data = data

# train_dataset = MultiStepInputDataset(train_data, history_steps=10)

# # 随机划分 80% train, 20% val
# train_size = int(0.8 * len(train_dataset))
# val_size = len(train_dataset) - train_size
# train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # 打印一个batch看看
# for inputs, targets in train_loader:
#     print(f"Input shape: {inputs.shape}, Target shape: {targets.shape}")
#     num_variables = inputs.shape[1]
#     break  # 只打印一次

In [3]:
import torch
import torch.nn as nn
import math


class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y  # Apply the attention weights

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=(kernel_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)  # Concatenate along channel dimension
        y = self.conv(y)
        return x * self.sigmoid(y)

class ChannelSpatialSELayer(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super().__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x


class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


class ResidualBlock(nn.Module):
    """A basic residual block with optional downsampling."""
    def __init__(self, in_channels, out_channels, stride=1, Separable=False, CBAM=False):
        super().__init__()
        
        conv = DepthwiseSeparableConv if Separable else nn.Conv2d
        self.cbam = ChannelSpatialSELayer(out_channels) if CBAM else nn.Identity()
        
        self.conv1 = conv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut for matching dimensions
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride) \
            if stride != 1 or in_channels != out_channels else nn.Identity()

    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x = (x + shortcut) / math.sqrt(2.)  # Residual connection
        x = self.cbam(x)  # Ensure CBAM has correct input channels
        return x


class Encoder(nn.Module):
    def __init__(self, input_channels, latent_dim, nx, ny, Separable=False, CBAM=False):
        super().__init__()
        self.nx, self.ny = nx, ny

        self.features = nn.Sequential(
            ResidualBlock(input_channels, 64, stride=2, Separable=Separable, CBAM=CBAM),
            ResidualBlock(64, 128, stride=2, Separable=Separable, CBAM=CBAM),
            ResidualBlock(128, 256, stride=2, Separable=Separable, CBAM=CBAM),
            ResidualBlock(256, 512, stride=2, Separable=Separable, CBAM=CBAM),
        )

        self.flatten_size = (nx // 16) * (ny // 16) * 512
        self.fc = nn.Linear(self.flatten_size, latent_dim)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        return self.fc(x)

class Decoder(nn.Module):
    def __init__(self, output_channels, latent_dim, nx, ny, CBAM=False):
        super().__init__()
        self.nx, self.ny = nx, ny

        self.fc = nn.Linear(latent_dim, 512 * (nx // 16) * (ny // 16))
        self.unflatten = nn.Unflatten(1, (512, nx // 16, ny // 16))

        self.features = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            ChannelSpatialSELayer(256) if CBAM else
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            ChannelSpatialSELayer(128) if CBAM else
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            ChannelSpatialSELayer(64) if CBAM else
            nn.ReLU(),
            nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1),
        )
    
    def forward(self, z):
        x = self.fc(z)
        x = self.unflatten(x)
        return self.features(x)


class Autoencoder(nn.Module):
    """Complete Autoencoder model."""
    def __init__(self, input_channels, latent_dim, nx, ny, Separable=False, CBAM=False):
        super().__init__()
        self.encoder = Encoder(input_channels, latent_dim, nx, ny, Separable=Separable, CBAM=CBAM)
        self.decoder = Decoder(input_channels, latent_dim, nx, ny)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


# # Test the model
# if __name__ == "__main__":
#     # Input dimensions
#     input_channels = 2  # For example, RGB=3, grayscale=1, etc.
#     latent_dim = 256
#     nx, ny = 64, 64  # Height and Width of input images

#     # Create autoencoder
#     autoencoder = Autoencoder(input_channels, latent_dim, nx, ny, Separable=True, CBAM=True).to("cuda:1")

#     # Test input
#     x = torch.randn(16, input_channels, nx, ny).to("cuda:1")  # Batch of 16 images
#     reconstructed = autoencoder(x)

#     # Ensure output shape matches input shape
#     print(f"Input shape: {x.shape}")
#     print(f"Reconstructed shape: {reconstructed.shape}")


In [4]:

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, n=1):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.n = n  # Default prediction steps
        self.cell = nn.LSTM(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            dropout=0, 
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, input_dim)
        self.activation = nn.ReLU()  # Use ReLU activation

    def forward(self, x, n=None):
        if n is None:  # Use default n if not provided
            n = self.n
        
        device = x.device
        batch_size = x.size(0)  # Get batch size
        seq_length = x.size(1) if len(x.shape) > 2 else 1  # Get sequence length
        
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, dtype=torch.float32, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, dtype=torch.float32, device=device)

        # Add time dimension if input is 2D
        if len(x.shape) == 2:
            x = x.unsqueeze(1)  # [batch_size, 1, input_dim]
        
        # Pass through LSTM
        outputs, (h, c) = self.cell(x, (h0, c0))
        
        # Apply ReLU activation to LSTM outputs
        outputs = self.activation(outputs)  # [batch_size, seq_length, hidden_dim]

        # Decode the last hidden state output
        y = [self.fc(outputs[:, -1, :])]  # [batch_size, input_dim]
        
        # Multi-step prediction
        for _ in range(1, n):
            last_output = y[-1].unsqueeze(1)  # [batch_size, 1, input_dim]
            outputs, (h, c) = self.cell(last_output, (h, c))
            outputs = self.activation(outputs)  # Apply ReLU activation
            y.append(self.fc(outputs[:, -1, :]))
        
        # Return last prediction and all predictions
        return y[-1], torch.stack(y, dim=1)  # [batch_size, n, input_dim]

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam


input_step = 10
pred_step = 1
num_epochs = 30
batch_size = 32
latent_dim = 512
device = "cuda:7" 
lr_min = 1e-5

train_data = data

train_dataset = MultiStepInputDataset(train_data, history_steps=10)

# 随机划分 80% train, 20% val
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 打印一个batch看看
for inputs, targets in train_loader:
    print(f"Input shape: {inputs.shape}, Target shape: {targets.shape}")
    num_variables = inputs.shape[2]
    break  # 只打印一次

autoencoder = Autoencoder(input_channels=num_variables, latent_dim=latent_dim, nx=128, ny=128, Separable=True, CBAM=True).to(device)
lstm = LSTM(input_dim=latent_dim, hidden_dim=latent_dim, num_layers=2, n=pred_step).to(device)

num_params = sum(p.numel() for p in autoencoder.parameters())
print(f"模型总参数量: {num_params / 1e6:.2f} M")


Input shape: torch.Size([32, 10, 2, 128, 128]), Target shape: torch.Size([32, 1, 2, 128, 128])
模型总参数量: 37.10 M


In [17]:

optimizer = Adam(list(autoencoder.parameters()) + list(lstm.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

num_epochs = 10
criterion = nn.MSELoss()

for epoch in range(num_epochs):
    autoencoder.train()
    lstm.train()

    total_loss = 0.0
    
    for batch_idx, (input_data, target_data) in enumerate(train_loader):
       
        input_data = input_data.to(device)  # Shape: [batch_size, input_step, num_variables, x_dim, y_dim]
        target_data = target_data.to(device)  # Shape: [batch_size, pred_step, num_variables, x_dim, y_dim]

       
        batch_size, input_step, num_variables, x_dim, y_dim = input_data.shape
        input_data_flat = input_data.view(-1, num_variables, x_dim, y_dim)  # Shape: [batch_size * input_step, num_variables, x_dim, y_dim]
        target_data_flat = target_data.view(-1, num_variables, x_dim, y_dim)

       
        latent_features = autoencoder.encoder(input_data_flat)  # Shape: [batch_size * input_step, latent_dim]
        latent_features = latent_features.view(batch_size, input_step, -1)  # Shape: [batch_size, input_step, latent_dim]
        _, lstm_outputs = lstm(latent_features, n=pred_step)  # Shape: [batch_size, pred_step, latent_dim]

        lstm_output_flat = lstm_outputs.view(-1, lstm_outputs.size(-1))  # Shape: [batch_size * pred_step, latent_dim]
        reconstructed = autoencoder.decoder(lstm_output_flat)  # Shape: [batch_size * pred_step, num_variables, x_dim, y_dim]
        
        
        loss = criterion(reconstructed, target_data_flat)
        total_loss += loss.item()

        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_loader)
    scheduler.step(avg_loss)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Reconstruction Loss: {avg_loss:.2e}")
    print(f"learning rate for lstm:{optimizer.param_groups[0]['lr']}")

    if optimizer.param_groups[0]['lr'] < lr_min:
        print(f"Learning rate below threshold ({lr_min}). Stopping early.")
        break    
       

    

print("Training complete!")

Epoch [1/10], Reconstruction Loss: 1.19e-04
learning rate for lstm:0.0001
Epoch [2/10], Reconstruction Loss: 1.14e-04
learning rate for lstm:0.0001
Epoch [3/10], Reconstruction Loss: 1.11e-04
learning rate for lstm:0.0001
Epoch [4/10], Reconstruction Loss: 1.10e-04
learning rate for lstm:0.0001
Epoch [5/10], Reconstruction Loss: 1.09e-04
learning rate for lstm:0.0001
Epoch [6/10], Reconstruction Loss: 1.04e-04
learning rate for lstm:0.0001
Epoch [7/10], Reconstruction Loss: 1.03e-04
learning rate for lstm:0.0001
Epoch [8/10], Reconstruction Loss: 1.03e-04
learning rate for lstm:0.0001
Epoch [9/10], Reconstruction Loss: 9.97e-05
learning rate for lstm:0.0001
Epoch [10/10], Reconstruction Loss: 9.57e-05
learning rate for lstm:0.0001
Training complete!


In [18]:

# torch.save(autoencoder.state_dict(), f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/lo_autoencoder_latent_{latent_dim}.pth")
# torch.save(lstm.state_dict(), f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/lo_lstm_latent_{latent_dim}.pth")
# print("Models saved successfully!")

Models saved successfully!


In [18]:
import torch
import numpy as np


x_test = np.load(f'/data3/chengjingwen/diffusion-mdpnet/data/lo/uv.npy')
x_test = torch.tensor(x_test, dtype=torch.float32) 



x_train = np.load(f'/data3/chengjingwen/diffusion-mdpnet/data/lo/uv.npy')
x_train = torch.tensor(x_train, dtype=torch.float32) 

# 计算训练集的 min 和 max，维度为 (1, n_vars, 1, 1)
xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最小值
xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  # 对轨迹数、时间步和空间维度求最大值

del x_train
x_test_normalized = (x_test - xmin) / (xmax - xmin)




print(f"Test data normalized: min={x_test_normalized.min().item()}, max={x_test_normalized.max().item()}")

Test data normalized: min=0.0, max=1.0


In [20]:
#### AE的自回归为什么这么差？？？ check

import numpy as np
import torch
from torchmetrics.functional import structural_similarity_index_measure as ssim

import matplotlib.pyplot as plt
from IPython.display import clear_output
pred_step = 1  
pred_steps = 20
latent_dim = 512
start_time = 10


def compute_nmse(predictions, targets):
    """计算 NMSE"""
    return torch.mean(((predictions - targets) ** 2)) / torch.mean((targets ** 2))

def compute_rmse(predictions, targets):
    """计算 RMSE"""
    mse = torch.mean((predictions - targets) ** 2)
    return torch.sqrt(mse)


def test_model(autoencoder, lstm, test_data, device):
    autoencoder.eval()
    lstm.eval()

    num_trajectories, T, num_variables, x_dim, y_dim = test_data.shape

    nmse_list = []
    ssim_list = []
    rmse_list = []

    
    for trajectory_idx in range(num_trajectories):
        trajectory = test_data[trajectory_idx]  # shape [100, num_variables, x_dim, y_dim]
        # start_time = np.random.randint(0, 40)  # 随机选起始时间
        
        targets = trajectory[start_time :start_time + pred_steps].to(device)  # 未来 50 步

        # 初始化预测结果
        predictions = torch.zeros((pred_steps, num_variables, x_dim, y_dim)).to(device)
        predictions_encoded = torch.zeros((pred_steps, latent_dim)).to(device)

        for t in range(0,pred_steps,pred_step):  
            if t == 0:
                # 起始输入通过 Autoencoder 压缩
                input_data = trajectory[start_time - 10: start_time].to(device)  # shape: [10, num_variables, x_dim, y_dim]
                latent = autoencoder.encoder(input_data).unsqueeze(0)  # shape: [1, 10, latent_dim]
            
            
                
                
            
            # LSTM 多步预测
            # print(latent[0,0])
            _, latent_preds = lstm(latent, n=pred_step)  # shape: [1, 1, latent_dim]
            latent_preds = latent_preds.squeeze(0)  # shape: [pred_step, latent_dim]
            latent = latent[0,1:]  # shape: [10, latent_dim]
            # print(latent.shape, latent_preds.shape)
            latent = torch.cat([latent, latent_preds], dim = 0).unsqueeze(0)
            
            predictions_encoded[t:t+pred_step] = latent_preds
            
            
            predictions[t:t+pred_step] = autoencoder.decoder(latent_preds)
            # ipdb.set_trace()
            
        if trajectory_idx == 0:
            for t in range(90):
                plt.figure(figsize=(12, 4))

                # Ground Truth
                plt.subplot(131)
                plt.title(f"Ground Truth (step={ t + 1})")
                plt.imshow(targets[t, 0].detach().cpu().numpy(), cmap="coolwarm", vmin=0, vmax=1) 
                plt.colorbar()

                # Prediction
                plt.subplot(132)
                plt.title("Prediction")
                plt.imshow(predictions[t, 0].detach().cpu().numpy(), cmap="coolwarm", vmin=0, vmax=1)  
                plt.colorbar()

                # Difference and MSE
                mse = torch.mean((targets[t, 0] - predictions[t, 0]) ** 2).item() 
                plt.subplot(133)
                plt.title(f"Difference (MSE={mse:.5f})")
                plt.imshow(torch.abs(targets[t, 0] - predictions[t, 0]).detach().cpu().numpy(), cmap="jet")
                plt.colorbar()

                plt.show()
                clear_output(wait=True)  
            

       
        ssim_values = ssim(predictions, targets, data_range=1.0).item() 
        nmse_trajectory = compute_nmse(predictions, targets).item() 
        rmse_trajectory = compute_rmse(predictions, targets).item() 

        nmse_list.append(nmse_trajectory)
        ssim_list.append(ssim_values)
        rmse_list.append(rmse_trajectory)

    nmse_mean, nmse_std = np.mean(nmse_list), np.std(nmse_list)
    ssim_mean, ssim_std = np.mean(ssim_list), np.std(ssim_list)
    rmse_mean, rmse_std = np.mean(rmse_list), np.std(rmse_list)

    return {
        "NMSE_mean": nmse_mean,
        "NMSE_std": nmse_std,
        "SSIM_mean": ssim_mean,
        "SSIM_std": ssim_std,
        "RMSE_mean": rmse_mean,
        "RMSE_std": rmse_std
    }


device = 'cuda:1' if torch.cuda.is_available() else 'cpu'


autoencoder = Autoencoder(input_channels=2, latent_dim=latent_dim, nx=128, ny=128, Separable=True, CBAM=True).to(device)
lstm = LSTM(input_dim=latent_dim, hidden_dim=latent_dim, num_layers=2, n=1).to(device)

autoencoder.load_state_dict(torch.load(f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/lo_autoencoder_latent_512.pth"))
lstm.load_state_dict(torch.load(f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/lo_lstm_latent_512.pth"))


test_data = x_test_normalized.to(device).float()[:1]  
print(test_data.shape)

results = test_model(autoencoder, lstm, test_data, device)


print(f"NMSE Mean: {results['NMSE_mean']}")
print(f"NMSE Std: {results['NMSE_std']}")
print(f"SSIM Mean: {results['SSIM_mean']}")
print(f"SSIM Std: {results['SSIM_std']}")
print(f"RMSE Mean: {results['RMSE_mean']}")
print(f"RMSE Std: {results['RMSE_std']}")



KeyboardInterrupt: 

In [38]:
model = lstm.to("cuda:3")
num_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {num_params / 1e6:.2f} M")

模型总参数量: 10.89 M


In [52]:
print(test_data.shape)

torch.Size([40, 50, 3, 64, 64])
